### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v.1.0.15'

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker'
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

!mkdir -p {docker_file_dir}
! chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.15'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip:
    - python-dotenv
    - azureml-sdk[notebooks,automl,explain] 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

FROM continuumio/miniconda3:4.5.12
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# basic system packages, plus gcc
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 ca-certificates curl git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH         

CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.15


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + image_version_string


docker_file_path
docker_image_name
docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.15'

'georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.15'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001 282 Feb 16 01:07 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 840 Feb 16 01:07 dockerfile_aml-sdk_docker_image_sdk.v.1.0.15


In [9]:
!echo docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache
!     docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache

docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.15 -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_aml-sdk_docker_image_sdk.v.1.0.15 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker --no-cache
Sending build context to Docker daemon  3.584kB
Step 1/12 : FROM continuumio/miniconda3:4.5.12
 ---> ae46c364060f
Step 2/12 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in d0f8322b17ba
Removing intermediate container d0f8322b17ba
 ---> ead0d979caff
Step 3/12 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2 ca-certificates curl git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 0f9efacfab55
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://security.debian.org/debian-security stretch/updates InRelease [94.3 kB]
Get:3 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:4 http://deb.debian.org/debian stretch Release [118 kB]
G

Selecting previously unselected package libtsan0:amd64.
Preparing to unpack .../15-libtsan0_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libtsan0:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package libubsan0:amd64.
Preparing to unpack .../16-libubsan0_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libubsan0:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package libcilkrts5:amd64.
Preparing to unpack .../17-libcilkrts5_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libcilkrts5:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package libmpx2:amd64.
Preparing to unpack .../18-libmpx2_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libmpx2:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package libquadmath0:amd64.
Preparing to unpack .../19-libquadmath0_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libquadmath0:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package libgcc-6-dev:amd64.
Preparing to unpack .../20-libgcc-6-dev_6.3.0-18+deb9u1_amd64.deb ...
Unpa

  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/4e/5b/1b/be8c8a830a0243af85b2946a0aece2c6743d7f7f946977ed67
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f


  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /root/.cache/pip/wheels/21/88/00/697879eab372b1071f60675adf91f6da0ae015313318b35421
  Stored in directory: /root/.cache/pip/wheels/ee/30/5c/3a3b5e1386c8db9a3be5f5c3933644ae0533c1351c6a8eb4b5
  Stored in directory: /root/.cache/pip/wheels/e8/74/51/42c2d41c02bdc6f0e604476b7e4293b8c98d0bcbfa1dff78c8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /root/.cache/pip/wheels/90/db/f8/2c3101f72ef1ad434e4662853174126ce30201a3e163dcbeca
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pathspec SecretStorage pyyaml dill shap JsonSir JsonForm gast termcolor absl-py pycparser
  Found existing installation: nump

Removing intermediate container 765562d5b405
 ---> 4d7a69993612
Step 8/12 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in 154dd9c01ea1
Removing intermediate container 154dd9c01ea1
 ---> d19166545958
Step 9/12 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in 6787bbd82fe7
Removing intermediate container 6787bbd82fe7
 ---> e397b0feed61
Step 10/12 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in 5284728e3d7a
Removing intermediate container 5284728e3d7a
 ---> 35c6fb1b0f07
Step 11/12 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 96bc30f2ea89
Removing intermediate container 96bc30f2ea89
 ---> 0286ffff4bde
Step 12/12 : CMD /bin/bash
 ---> Running in 84097139c40c
Removing intermediate container 84097139c40c
 ---> 6cf3c3e794cf
Successfully built 6cf3c3e794cf
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.15


In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 0


In [11]:
!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/

#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [12]:
# !docker login -u=$docker_login -p=
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

b5e6b557: Preparing 
e14c0d4c: Preparing 
c2f12164: Preparing 
6384d46a: Preparing 
8fbc5764: Preparing 
828fac1a: Preparing 
ae060f2d: Preparing 


b5e6b557: Pushing  1.897GB/2.406GBraccount/aml-sdk_docker K

b5e6b557: Pushed   2.454GB/2.406GBsdk.v.1.0.15: digest: sha256:36745108eb7dcd8b068377204e174392a53cdcd8cfff04158bdc29aa5ccbbcfd size: 2002


### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [13]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.15 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [14]:
!echo docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

docker run georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.15 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.15


#### Do the same in a  notebook

In [15]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [16]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [17]:
!ls -l $crt_local_dir

total 4
-rw-rw-r-- 1 loginvm001 loginvm001 982 Feb 16 01:14 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [18]:
!docker run -v $crt_local_dir:/workspace:rw $docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 979 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [19]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.15\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


In [20]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 350691 bytes to createAMLSDKDocker.html
